## Import các thư viện

In [ ]:
import matplotlib.pyplot as plt
from keras_nlp.api.models import DebertaV3Preprocessor
from keras_nlp.api.models import DebertaV3Classifier
import keras
import tensorflow as tf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split

## Các tham số

In [ ]:
# NOTE: Tham khảo : https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter#%F0%9F%8D%BD%EF%B8%8F-%7C-Preprocessing
# NOTE: đọc hết TODO

# TODO: đổi path file csv
data_path = ".csv"
# Sử dụng model DeBERTa-V3-Extra-Small-English
model_preset = "deberta_v3_extra_small_en"
max_epochs = 10
batch_size = 8
shufle = 1000
lr = 0.0000003

## Các hàm chính để xử lý model

### 1. Tiền xử lý dữ liệu

 Label giá trị trị từ 1-6 (cột score) là dữ liệu categorical.
 Nhưng để làm bài này, label phải là dữ liệu ordinary
 Biến đổi label CATEGORICAL SANG ORDINARY bằng cách :

 Encode label thành một vector nhị phân có độ dài 6, bit i bằng 1 nghĩa là đã thỏa yêu cầu i
 Các giá trị trong vector CHO BIẾT XÁC SUẤT VỊ TRÍ i THỎA YÊU CẦU i.
 Từ đó có thể sử dụng CROSS-ENTROPY để sử dụng hàm lỗi so sánh sự tương đồng giữa 2 phân phối.

 Từ trên ta tạo tính ORDINARY như sau:

 Bài được chấm điểm 1 <= i <= 6 nghĩa là đã thỏa mãn yêu cầu i, VÀ CŨNG THỎA TẤT CẢ CÁC YÊU CẦU TRƯỚC ĐÓ (<= i),
 nên các bit trong vector nhãn sẽ = 1 ở các vị trí <= i, và 0 ở các vị trí còn lại.
 Cách làm này giúp có TÍNH SO SÁNH, vì một khi được điểm j >= i, thì các bit 1 của điểm i cũng = 1 ở trong j


In [ ]:
def make_ordinary(y):
    n = len(y)
    z = np.zeros((n, 6), "float32")
    for i in range(n):
        s = y[i]
        z[i, :s] = 1
    return z

In [ ]:
#TODO: code tiền xử lý đầu vào

### 2. Tạo và cấu hình model

Load model đã được cấu hình và train sẵn, sau đó train để fine-tune model này phù hợp cho bài toán. </br>
Như đã nói trên sử đầù ra là một vector độ dài 6, mỗi vị trí cho biết xác suất thỏa yêu cầu i.</br>
Đầu ra của mạng sử dụng sigmoid cho ra giá trị 0->1, chuẩn hóa thành xác suất cho mỗi vị trí i.</br>
Sử dụng hàm Binary Cross-Entropy, vì ta tính xác suất mỗi vị trí (mỗi vị trí đúng (1) đến sai (0)),
sau đó keras.losses.BinaryCrossentropy tổng hợp Loss tại mỗi vị trí làm Loss chung
Sử dụng Adam optimizer kết hợp SGD và momentum

In [ ]:
def create_DebertaV3_model(model_preset, manual_preprocess=False):
    if not manual_preprocess:
        debertaV3 = DebertaV3Classifier.from_preset(model_preset, num_classes=6)
    else:
        debertaV3 = DebertaV3Classifier.from_preset(
            model_preset, preprocessor=None, num_classes=6
        )
    inputs = debertaV3.input
    outputs = debertaV3(inputs)

    prob_outputs = keras.layers.Activation("sigmoid")(outputs)
    model = keras.Model(inputs, prob_outputs)

    return model

### 3. Tạo dataset cho dữ liệu train và validation

Load dữ liệu từ file csv và tạo tf.data.Dataset cho dữ liệu train và dữ liệu validation.</br>
Dataset load dữ liệu theo lô (và cache để tiết kiệm bộ nhớ trong) và hoán vị dữ liệu sau mỗi vòng lặp cho THUẬT TOÁN SGD

In [ ]:
def create_dataset():
    df = pd.read_csv(data_path)
    X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(
        df["full_text"], df["score"], test_size=0.2, stratify=df["score"]
    )
    X_train = X_train_df.tolist()
    X_val = X_val_df.tolist()
    y_train = make_ordinary(y_train_df.tolist())
    y_val = make_ordinary(y_val_df.tolist())

    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .cache()
        .shuffle(shufle, seed=0)
        .batch(batch_size)
    )
    opt = tf.data.Options()
    opt.experimental_deterministic = False
    train_ds = train_ds.with_options(opt)

    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val)).cache().batch(batch_size)
    )

    return train_ds, val_ds

## Huấn luyện model

In [ ]:
train_ds, val_ds = create_dataset()
model = create_DebertaV3_model(model_preset)

model.compile(
    optimizer=keras.optimizers.Adam(lr),
    loss=keras.losses.BinaryCrossentropy(),
)

save_model = keras.callbacks.ModelCheckpoint(
    "model.h5",  # HDF5
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="min",
)

# Train model
history = model.fit(
    train_ds,
    epochs=max_epochs,
    validation_data=val_ds,
    callbacks=[save_model],
)

## Thống kê lỗi và đánh giá

In [ ]:
training_loss = history.history["loss"]
validation_loss = history.history["val_loss"]

epoch_rows = range(1, len(training_loss) + 1)
plt.plot(epoch_rows, training_loss, "r--")
plt.plot(epoch_rows, validation_loss, "b-")
plt.legend(["Training Loss", "Validation Loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.savefig("Loss_plot.png")
plt.show()